# 1. Pytorch → onnx (Done)

In [4]:
import os

os.chdir('../')

import torch

from latency_predictor.arch_utils import *

from nas.supernet.supernet_yolov7 import YOLOSuperNet

device = 'cpu'
supernet = YOLOSuperNet(cfg='./yaml_config/yolov7_supernet.yml').to(device)

supernet.set_active_subnet([5,5,5,5,7,7,7,1])
subnet = supernet.get_active_subnet()
b_arch, h_arch, save = split_backbone_head(subnet)
backbone = skin_backbone(b_arch, save).eval()
head = skin_head(h_arch, save).eval()

img = torch.rand(1, 3, 640, 640).to(device)
torch.onnx.export(backbone,
                  img,
                  'exported_models/backbone_5_5_5_5.onnx',
                  export_params=True,
                  verbose=False,
                  opset_version=10,        
                  do_constant_folding=False,
                  input_names = ['input'],
                  output_names = ['output'])

img = torch.rand(1, 1024, 20, 20).to(device)
torch.onnx.export(head,
                  img,
                  'exported_models/head_7_7_7_1.onnx',
                  export_params=True,
                  verbose=False,
                  opset_version=10,
                  do_constant_folding=False,
                  input_names = ['input'],
                  output_names = ['output'])

[0, 1, 3, 5]


IndexError: list index out of range

In [ ]:
from onnxsim import simplify
import onnx
